In [ ]:
import julia
julia.install()

from pathlib import Path
# %load_ext julia.magic
# %julia using Pkg
# from julia import Main
# from julia import Pkg

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

# # Pkg.add("AdaptiveResonance")
# Pkg.activate(str(projectdir()))

# from julia import AdaptiveResonance as ART
# art = ART.DDVFA()

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()
# Take the first 500 data points: it's hard to see 1500 points
X = digits.data[:500]
y = digits.target[:500]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
from julia import Main as j

j.project_dir = projectdir()
j.eval("using Pkg; Pkg.activate(project_dir)")
j.eval("using AdaptiveResonance")
j.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
print("Loaded DDVFA")

# ART.train!(art, X, y=y)
j.X_train = X_train.transpose()
j.y_train = y_train
j.X_test = X_test.transpose()
# j.eval("X_train_local = Matrix{Float64}(X_train)")
# j.eval("y_train_local = Vector{Int32}(y_train)")
j.eval("art.config = DataConfig(0.0, 16.0, 64)")
j.eval("train!(art, X_train, y=y_train)")
y_hat = j.eval("classify(art, X_test)")
# y_hat

n_weights = j.eval("AdaptiveResonance.get_n_weights(art)")
print(n_weights, X_train.shape)
print(accuracy_score(y_test, y_hat))